##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Quickstart with Python

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/tutorials/python_quickstart"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Google AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This quickstart demonstrates how to use the Python SDK for the Gemini API, which gives you access to Google's Gemini large language models. In this quickstart, you will learn how to:

1. Set up your development environment and API access to use Gemini.
2. Generate text responses from text inputs.
3. Generate text responses from multimodal inputs (text and images).
4. Use Gemini for multi-turn conversations (chat).
5. Use embeddings for large language models.

## Prerequisites

You can run this quickstart in [Google Colab](https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb), which runs this notebook directly in the browser and does not require additional environment configuration.

Alternatively, to complete this quickstart locally, ensure that your development environment meets the following requirements:

-  Python 3.9+
-  An installation of `jupyter` to run the notebook.

## Setup

### Install the Python SDK

The Python SDK for the Gemini API, is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:

In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 2.7 MB/s eta 0:00:00


### Import packages

Import the necessary packages.

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Used to securely store your API key
from google.colab import userdata

### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`

In [ ]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## List models

Now you're ready to call the Gemini API. Use `list_models` to see the available Gemini models:

* `gemini-pro`: optimized for text-only prompts.
* `gemini-pro-vision`: optimized for text-and-images prompts.

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


Note: For detailed information about the available models, including their capabilities and rate limits, see [Gemini models](https://ai.google.dev/models/gemini). There are options for requesting [rate limit increases](https://ai.google.dev/docs/increase_quota). The rate limit for Gemini-Pro models is 60 requests per minute (RPM).

The `genai` package also supports the PaLM  family of models, but only the Gemini models support the generic, multimodal capabilities of the `generateContent` method.

## Generate text from text inputs

For text-only prompts, use the `gemini-pro` model:

In [ ]:
model = genai.GenerativeModel('gemini-pro')

The `generate_content` method can handle a wide variety of use cases, including multi-turn chat and multimodal input, depending on what the underlying model supports. The available models only support text and images as input, and text as output.

In the simplest case, you can pass a prompt string to the <a href="https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content"><code>GenerativeModel.generate_content</code></a> method:

In [ ]:
%%time
string="""
Summarize the below text in Bangla  ( A phonecall). Provide key points of this conversation:
----------------------------------------------------------------------------------------------------
SPEAKER_00: হ্যালো মিরাজ।
SPEAKER_01: হ্যাঁ তামিম ভাই। আসালামালাইকুম।
SPEAKER_00: কী অবস্থায় এটা কেমন আছে?
SPEAKER_01: এই তো আল্লাহ মুজিবুল্লাহ ভাই আপনি কেমন আছেন?
SPEAKER_00: কেমন থাকি রে তুই? এই মুশফিকের কোনো কাজ করছিস রে মিরাজ?
SPEAKER_01: ভাই শুনলাম আমিও। সে জিনিসটা ঠিক আছে। এখন কী করবেন?
SPEAKER_00: লাস্টে আমাদের ফাইনালের পরে আমাদের মিটিংয়ে কী কথা হয়েছে? আমি তোর ওই সময় একটা কথা বলছিলাম না যে, আমরা চেষ্টা করব যে সত্তর থেকে আশি পারসেন্ট আমরা যেরকম সেইম টিম থাকতে পারি। আর এই কথার বেইজিতও ছিল তুই মুশফিকার সবসময় ঠিক না। মুশফিক আমার জন্য একটা ইম্পটেন্ট একজন ছিল। সে সে করছেটা কি? শুন সে আমার সাথে তো কথা বলেই নেই। বলে নেই। সে বলে এখন আলাদা একটা টিম বানাচ্ছে।
SPEAKER_01: আমি একটু হালকা একটু শুনেছিলাম বাটা হান্ডেট পারসেন্ট কনফার্ম ছিলাম না। বাট আপনি এখন যেটা বললেন, আমি তো এখন হান্ডেট পার্সেন্ট শিরোলাম যে ভাই মনে হয়।
SPEAKER_00: বিশ্ববিদ্যালয়ের মধ্যে একটি বিদ্যালয়ের মধ্যে একটি বিদ্যালয়ের মধ্যে একটি বিদ্যালয়ের।
SPEAKER_01: আলাদা টিম বানাচ্ছে।
SPEAKER_00: তুই দেখছস আমি ওরে কি ধরনের ফ্রিডম দিছি পুরা টিমে। আমি ওরে দায়িত্ব দিছি পুরা টিম চালানোর। খেলা শেষ হয়েছে। খেলা শেষে আমি সবচেয়ে বেশি ওরে প্রেস করছি। আমি এখন পর্যন্ত ওরে প্রেস থাকি যে সে পুরা টিমটা হ্যান্ডেল করছে। তার কন্ট্রোবেশনটা কত বেশি। সে এই মুহূর্তে আমাকে এরকম মাঝপথে ছেড়ে যাওয়াটা কি উচিত ছিল?
SPEAKER_01: এখন কী করবেন ভাই? এখন ভাই গেছে। এখন সে তো আপনারই ফ্রেন্ড।
SPEAKER_00: এখন আসলে তো তোদের সাথে নাই। খেলি না তো ন্যাশনাল টিমে এখন অনেকের অনেক ভাবটা বেড়ে গেছে। এটা হল সমস্যা। ঠিক আছে। কারণ হল যে, এখন তোদের সাথে ক্যাপ্টেন থাকলে, ক্যাপ্টেন থাকলে তো এরকম আর তোরা করতে পারতি না। এখন আমার দাম নেই। এখন দাম কমে যাচ্ছে দেখে তোরা এসব করতেছে।
SPEAKER_01: ভাই শোনেন রাগ করেন না প্লিজ। কারণ কি ভাই আপনি তো জানেন মুশফিক বা সম্পর্কে সে তো একটু ইমোশনাল। হয়তো ইমোশনাল হয়ে পড়ে যায়।
SPEAKER_00: একবার আমাকে অ্যাটলিস বলত, চলে যাওয়ার আগে অ্যাটলিস। তখন তো আমার একটু শান্তি হইত। যেন আগে অ্যাটলিস বলেই গেছে, অসুবিধা না মিরাজ। সময় আমারও তো আসবে। পৃথিবী গোল। কিন্তু শুনে রাখ মিরাজ, একটা কথা শুন, পৃথিবীর গোল। আজকে তুই ওই সাইট, আমি এ সাইট। কালকে আমিও সাইডে বসব। তুই এ সাইডে আসবি। ভুলে যাস না। আর রোডা তোর বড় ভাইরেও বলে দিস।
SPEAKER_01: ভাই আপনারা রাগ করেন না। ঠান্ডা হন ভাই।
SPEAKER_00: তুই দেখ আমি কি করি। আমি কথা করব না তোর সাথে এ ব্যাপারে কথা করব। জিনিসটা কাজটাও কোনোভাবেই ঠিক করেনি।
SPEAKER_01: থান্ডার।
SPEAKER_00: ঠিক করা।
SPEAKER_01: দরকার নাই।
"""

response = model.generate_content(string)

CPU times: user 320 ms, sys: 33.1 ms, total: 353 ms
Wall time: 23 s


In simple cases, the `response.text` accessor is all you need. To display formatted Markdown text, use the `to_markdown` function:

In [ ]:
to_markdown(response.text)

> **ফোন কলের সারাংশ**
> 
> **মূল পয়েন্টসমূহ:**
> 
> * মিরাজ মুশফিকের সাথে কাজ না করার বিষয়ে আলোচনা করছে।
> * মুশফিক আলাদা দল গড়ছে বলে মিরাজের অভিযোগ।
> * মিরাজ মনে করছে মুশফিক অনুচিতভাবে দল ত্যাগ করেছে।
> * মিরাজ জাতীয় দলের অন্যান্য খেলোয়াড়দের অসন্তুষ্টি এবং সম্মানের অভাব সম্পর্কে কথা বলছে।
> * মিরাজ মিরাজকে সতর্ক করেছে যে ভবিষ্যতে রোল পরিবর্তন হতে পারে।
> 
> **বাংলায় সারাংশ:**
> 
> টামিম: হ্যালো মিরাজ।
> 
> মিরাজ: হ্যাঁ ভাই, আসসালামু আলাইকুম।
> 
> টামিম: কেমন আছো?
> 
> মিরাজ: আল্লাহর রহমতে ভালো, আপনি কেমন আছেন?
> 
> টামিম: কেমন থাকবোরে তুমি? মুশফিকের সাথে কাজ করছো এখন?
> 
> মিরাজ: ভাই, শুনেছি। বিষয়টা সত্যি। এখন কী করা যাবে?
> 
> টামিম: আমাদের আগের মিটিংয়ে কী কথা হয়েছিল মনে আছে? আমরা বলেছিলাম যে, সত্তর থেকে আশি শতাংশ আগের দলটাকে রাখতে চাই। আর এর কারণ ছিল মুশফিকের অভাব। মুশফিক আমার জন্য খুব গুরুত্বপূর্ণ ছিল। কিন্তু সে এখন কী করছে? সে আমার সাথে কথাও রাখছে না। সে বলছে আলাদা দল তৈরি করবে।
> 
> মিরাজ: আমি কিছুটা শুনেছিলাম কিন্তু নিশ্চিত ছিলাম না। কিন্তু আপনি এখন যেটা বললেন, আমি এখন শতভাগ নিশ্চিত যে ভাই, হয়তো মনে হচ্ছে।
> 
> টামিম: একটু একটু করে সবকিছু হবে।
> 
> মিরাজ: আলাদা দল বানাচ্ছে?
> 
> টামিম: তুমি দেখেছো আমি কী রকম স্বাধীনতা দিয়েছি পুরো দলজুড়ে। আমি তাকে সম্পূর্ণ দল পরিচালনার দায়িত্ব দিয়েছি৷ খেলা শেষ হয়ে গেছে৷ খেলা শেষে আমি সবচেয়ে বেশি তাকে চাপ দিচ্ছি৷ আমি এখনও তাকে চাপে রাখি যে সে পুরো দলটাকে নিয়ন্ত্রণ করছে৷ তার কতটা নিয়ন্ত্রণ আছে৷ এই মুহূর্তে আমাকে এভাবে মাঝপথে ফেলে যাওয়াটা কি তার উচিত ছিল?
> 
> মিরাজ: এখন কী করবেন ভাই? এখন ভাই গেছেন৷ সে আপনার বন্ধু।
> 
> টামিম: এখন তোমাদের সাথে নেই৷ জাতীয় দলে খেলি না, এখন অনেকের অনেক অনুভূতি বেড়ে গেছে৷ এটাই সমস্যা৷ কারণ, এখন তোমাদের সাথে ক্যাপ্টেন থাকলে, তোমরা এমনটা করতে পারবে না৷ এখন আমার কোনো মূল্য নেই৷ এখন মূল্য কমে যাচ্ছে দেখে তোমরা এসব করছো৷
> 
> মিরাজ: ভাই, দয়া করে রাগ করবেন না৷ কারণ ভাই, আপনি জানেন মুশফিক একটু বেশি আবেগপ্রবণ৷ হয়তো আবেগে পড়ে যায়৷
> 
> টামিম: আগে আমাকে অ্যাটলিস বলত, চলে যাওয়ার আগে অ্যাটলিস৷ তখন আমার কিছুটা শান্তি হতো। যেন আগে অ্যাটলিস বলেই গেছে, অসুবিধা নেই মিরাজ, সময় আমারও আসবে৷ পৃথিবী গোল৷ কিন্তু মিরাজ, একটা কথা শোনো, পৃথিবী গোল৷ আজকে তুমি ওই পাশে, আমি এই পাশে৷ কালকে আমিও ওই পাশে বসব৷ তুমি এ পাশে আসবে৷ ভুলে যো না৷ আর রোদা তোমার বড় ভাইকেও বলে দিও৷
> 
> মিরাজ: ভাই, আপনি রাগ করবেন না৷ শান্ত হোন ভাই৷
> 
> টামিম: তুমি দেখো আমি কী করি৷ আমি তোমার সাথে কথা বলব না৷ এ ব্যাপারে কথা বলব৷ এই জিনিসটাও ঠিক করোনি৷
> 
> মিরাজ: শান্ত হোন৷
> 
> টামিম: ঠিক করতে হবে৷
> 
> মিরাজ: দরকার নেই৷

If the API failed to return a result, use `GenerateContentRespose.prompt_feedback` to see if it was blocked due to safety concerns regarding the prompt.